In [1]:
import os

import numpy as np
import pandas as pd
import torch
from torch import device, cuda

from flyvis.examples.flyvision_ans import DECODING_CELLS
from from_retina_to_connectome_funcs import from_retina_to_model

device_type = "cuda" if cuda.is_available() else "cpu"
DEVICE = device(device_type)
batch_size = 10
last_good_frame = 8

/home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [2]:
# this needs to come from the previous step in the pipeline
# get data
activations_dir = "flyvis/parsed_objects"
activations = np.load(os.path.join(activations_dir, "decoding_activations.npy"), allow_pickle=True)
# labels = np.load(os.path.join(activations_dir, "decoding_labels.npy"), allow_pickle=True)
# toy labels as a tensor with all 1
# fixme!
labels = torch.ones(activations.shape[0], dtype=torch.long, device=DEVICE)

# this is probably here to stay
classification = pd.read_csv("adult_data/classification_clean.csv")

In [3]:
loader = from_retina_to_model(activations, labels, DECODING_CELLS, last_good_frame, DEVICE, batch_size)

100%|██████████| 34/34 [00:09<00:00,  3.52it/s]


In [4]:
# get a dataframe indicating which neurons will be used to classify
rational_neurons = pd.read_csv("adult_data/rational_neurons.csv", index_col=0)
decision_making_vector = torch.tensor(rational_neurons.values.squeeze(), dtype=torch.float16, device=DEVICE).detach()

In [5]:
from torch import autocast
from tqdm import tqdm
from torch.nn import BCEWithLogitsLoss
from torch.cuda.amp import GradScaler
from graph_models import GNNModel

# Initialize the model
model = GNNModel(num_node_features=1, decision_making_vector=decision_making_vector).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scaler = GradScaler()

# Initialize the loss function
criterion = BCEWithLogitsLoss()

model.train()
for batch_idx, batch in tqdm(enumerate(loader)):
    batch = batch.to(DEVICE)

    optimizer.zero_grad()
    
    with autocast(device_type):
        out = model(batch)
        loss = criterion(out, batch.y.unsqueeze(-1).float())    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

10it [00:08,  1.17it/s]
